In [3]:
## Instalando dependencias 
! pip install --no-cache-dir -r requirements.txt

In [1]:
## Importando os class
from functions.trino_provider import TrinoProvider
from functions.s3_functions import s3Function
from functions.format_data import FormatData
import pandas as pd
import csv

In [2]:
## Inicializando 
trinoProvider = TrinoProvider()
s3 = s3Function()
fmt = FormatData()

In [68]:
## Declarando variaveis
BUCKET_NAME='apresentacao'
CATALOG='minio'
SCHEMA='dev'

In [69]:
# MINIO

## Lista  de buckets
s3.get_buckets()

['apresentacao']

In [41]:
## Criando um bucket
s3.create_bucket(BUCKET_NAME)

Creating new bucket: apresentacao2


'Bucket: apresentacao2 created!'

In [73]:
## Lendo arquivo local e inserindo no bucket

orders = pd.read_csv('../files/orders.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/row/orders/orders.csv', data=orders, file_format='csv')

ingredients = pd.read_csv('../files/ingredients.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/row/ingredients/ingredients.csv', data=ingredients, file_format='csv')

items = pd.read_csv('../files/items.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/row/items/items.csv', data=items, file_format='csv')

recipe = pd.read_csv('../files/recipe.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/row/recipe/recipe.csv', data=recipe, file_format='csv')

{'data': b'row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out\n1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out\n2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in\n3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out\n4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out\n5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out\n6,ORD006,2024-02-12 07:28:20,It001,1,Morgan,in\n7,ORD006,2024-02-12 07:28:20,It016,1,Morgan,in\n8,ORD007,2024-02-12 07:33:58,It005,1,Riley,out\n9,ORD007,2024-02-12 07:33:58,It020,1,Riley,\n10,ORD008,2024-02-12 07:39:02,It006,1,Cameron,in\n11,ORD008,2024-02-12 07:39:01,It018,1,Cameron,\n12,ORD009,2024-02-12 07:44:02,It023,1,Quinn,out\n13,ORD009,2024-02-12 07:44:03,It011,1,Quinn,\n14,ORD010,2024-02-12 07:49:04,It024,1,Peyton,out\n15,ORD010,2024-02-12 07:49:05,It014,1,Peyton,\n16,ORD011,2024-02-12 07:53:06,It003,1,Brooke,out\n17,ORD012,2024-02-12 07:58:07,It007,1,Blake,out\n18,ORD013,2024-02-12 08:03:08,It009,1,Charlie,in\n19,ORD013,2024-02-12 08:03:09,It021,1,Charlie,\n20,ORD01

In [74]:
## Lista de objetos no bucket
s3.list_objects(BUCKET_NAME, dir=SCHEMA)

['dev/row/ingredients/ingredients.csv',
 'dev/row/items/items.csv',
 'dev/row/orders/orders.csv',
 'dev/row/recipe/recipe.csv']

In [75]:
orders = s3.load_file_s3(path='dev/row/orders/orders', bucket_name=BUCKET_NAME, format='csv')

In [58]:
## Retorno em df
## Exemplo de como ler arquivo direto do bucket(MinIo) 
orders = s3.load_file_s3(path='dev/row/orders/orders', bucket_name=BUCKET_NAME, format='csv')
items = s3.load_file_s3(path='dev/row/items/items', bucket_name=BUCKET_NAME, format='csv')
ingredients = s3.load_file_s3(path='dev/row/ingredients/ingredients', bucket_name=BUCKET_NAME, format='csv')
recipe = s3.load_file_s3(path='dev/row/recipe/recipe', bucket_name=BUCKET_NAME, format='csv')

error load_from_s3. S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /apresentacao/dev/row/orders/orders.csv, request_id: 17CB25FA3DD2BD13, host_id: e0c385c033c4356721cc9121d3109c9b9bfdefb22fd2747078acd22328799e36, bucket_name: apresentacao, object_name: dev/row/orders/orders.csv
error load_from_s3. S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /apresentacao/dev/row/items/items.csv, request_id: 17CB25FA3E7D3170, host_id: e0c385c033c4356721cc9121d3109c9b9bfdefb22fd2747078acd22328799e36, bucket_name: apresentacao, object_name: dev/row/items/items.csv
error load_from_s3. S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /apresentacao/dev/row/ingredients/ingredients.csv, request_id: 17CB25FA3F07A4F1, host_id: e0c385c033c4356721cc9121d3109c9b9bfdefb22fd2747078acd22328799e36, bucket_name: apresentacao, object_name: dev/row/ingredients/ingredients.csv
error load_f

In [77]:
# TRINO
trinoProvider.test_connection()

[['trino', 'http://192.168.176.4:8080', '351', True, 'active']]

In [78]:
## Exemplos de chamadas 

## Ver catalago de dados
cursor = trinoProvider.conn()
cursor.execute('SHOW CATALOGS')
print(cursor.fetchall())

trinoProvider.get_result_pandas('SHOW CATALOGS')


[['memory'], ['minio'], ['system'], ['tpcds'], ['tpch']]

Execute SQL: SHOW CATALOGS


,Catalog
0,memory
1,minio
2,system
3,tpcds
4,tpch


In [79]:
## Ver schemas no Trino
trinoProvider.get_result_pandas("SHOW SCHEMAS FROM minio LIKE 'dev'")


Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'


,Schema
0,dev


In [83]:
## lista de tabelas
trinoProvider.get_result_pandas("SHOW tables FROM minio.dev")


Execute SQL: SHOW tables FROM minio.dev


,Table
0,orders


In [84]:
## Deletando uma tabela
trinoProvider.get_result_pandas("DROP TABLE minio.dev.orders")


Execute SQL: DROP TABLE minio.dev.orders


,result
0,True


In [85]:
## Criando tabela no trino
key_s3= f"{SCHEMA}/row/orders"
trinoProvider.create_table_and_metadata(df=orders, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA,  key_s3=key_s3, table='orders', separator=',', format_file='csv')


NAME:  row_id  TYPE:  int64
NAME:  order_id  TYPE:  object
NAME:  created_at  TYPE:  object
NAME:  item_id  TYPE:  object
NAME:  quantity  TYPE:  int64
NAME:  cust_name  TYPE:  object
NAME:  in_or_out  TYPE:  object

Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'
Schema dev exist

Execute SQL: SHOW TABLES FROM minio.dev LIKE 'orders'
Creating table minio.dev.orders ...

Execute SQL: CREATE TABLE IF NOT EXISTS minio.dev.orders (
                        row_id  integer, 
order_id  varchar, 
created_at  varchar, 
item_id  varchar, 
quantity  integer, 
cust_name  varchar, 
in_or_out  varchar
                    )
                    WITH (
                        format = 'TEXTFILE',
                        skip_header_line_count = 1,
                        textfile_field_separator = ',',
                        external_location = 's3a://apresentacao//dev/row/orders'
                    )
                    
Table minio.dev.orders criada, file format csv 


""


In [ ]:
## Criando tabela no trino
key_s3= f"{SCHEMA}/row/items"
trinoProvider.create_table_and_metadata(df=items, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA, key_s3=key_s3, table='items', separator=',', format_file='csv')

In [ ]:
## Criando tabela no trino
key_s3 = f"{SCHEMA}/row/ingredients"
trinoProvider.create_table_and_metadata(df=ingredients, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA, key_s3=key_s3, table='ingredients', separator=',', format_file='csv')

In [ ]:
## Criando tabela  no trino
key_s3 = f"{SCHEMA}/row/recipe"
trinoProvider.create_table_and_metadata(df=recipe, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA, key_s3=key_s3, table='recipe', separator=',', format_file='csv')

In [89]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.orders").head()


Execute SQL: SELECT * FROM minio.dev.orders


,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out
3,4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out
4,5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out


In [90]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.items").head()


Execute SQL: SELECT * FROM minio.dev.items


,item_id,sku,item_name,item_cat,item_size,item_price
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45
3,It004,HDR-LAT-LG,Latte,Hot Drinks,Large,3.75
4,It005,HDR-FLT,Flat White,Hot Drinks,,3.15


In [82]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.ingredients").head()


Execute SQL: SELECT * FROM minio.dev.ingredients


,ing_id,ing_name,ing_weight,ing_meas,ing_price
0,ING001,Espresso beans,1000,grams,12.00
1,ING002,Whole Milk,1000,ml,1.20
2,ING003,Cheddar,500,grams,7.45
3,ING004,Mozzarella,500,grams,5.00
4,ING005,Whipped cream,300,ml,1.35


In [92]:
sql = """
    SELECT o.order_id
         , o.cust_name
        , i.item_name
        , i.item_size
        , i.item_price
      FROM minio.dev.orders as o
     INNER JOIN minio.dev.items as i
       ON o.item_id = i.item_id
"""
order_items = trinoProvider.get_result_pandas(sql)


Execute SQL: 
    SELECT o.order_id
         , o.cust_name
        , i.item_name
        , i.item_size
        , i.item_price
      FROM minio.dev.orders as o
     INNER JOIN minio.dev.items as i
       ON o.item_id = i.item_id



In [93]:
## Inserindo modificacao
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/bronze/order_items/order_items', data=order_items, file_format='csv')

{'data': b'order_id,cust_name,item_name,item_size,item_price\nORD001,Alex,Espresso,,2.15\nORD002,Jordan,Hot Chocolate,Large,4.6\nORD003,Taylor,Espresso,,2.15\nORD004,Casey,Iced Tea,Medium,3.25\nORD005,Jamie,Sandwich Salami&Mozzarella,,5.5\nORD006,Morgan,Cappuccino,Medium,3.45\nORD006,Morgan,Cold Coffee,Large,3.75\nORD007,Riley,Flat White,,3.15\nORD007,Riley,Iced Tea,Large,3.55\nORD008,Cameron,Caramel Macchiato,Medium,4.2\nORD008,Cameron,Cold Mocha,Large,4.6\nORD009,Quinn,Sandwich Ham&Cheese,,5.6\nORD009,Quinn,White Mocha,Medium,4.5\nORD010,Peyton,Sandwich Salami&Mozzarella,,5.5\nORD010,Peyton,Hot Chocolate,Large,4.6\nORD011,Brooke,Latte,Medium,3.45\nORD012,Blake,Caramel Macchiato,Large,4.6\nORD013,Charlie,Mocha,Medium,4.0\nORD013,Charlie,Lemonade,Medium,3.35\nORD014,Dakota,White Mocha,Large,4.7\nORD014,Dakota,Lemonade,Large,3.75\nORD015,Emerson,Latte,Large,3.75\nORD016,Finley,Cappuccino,Large,3.75\nORD017,Harper,Cappuccino,Medium,3.45\nORD018,Jordan,Hot Chocolate,Large,4.6\nORD019,Kend

In [94]:
## Criando tabela da transfomacao no trino
key_s3=f"{SCHEMA}/bronze/order_items"
trinoProvider.create_table_and_metadata(df=order_items, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA,  key_s3=key_s3, table='order_items', separator=',', format_file='csv')

NAME:  order_id  TYPE:  object
NAME:  cust_name  TYPE:  object
NAME:  item_name  TYPE:  object
NAME:  item_size  TYPE:  object
NAME:  item_price  TYPE:  float64

Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'
Schema dev exist

Execute SQL: SHOW TABLES FROM minio.dev LIKE 'order_items'
Creating table minio.dev.order_items ...

Execute SQL: CREATE TABLE IF NOT EXISTS minio.dev.order_items (
                        order_id  varchar, 
cust_name  varchar, 
item_name  varchar, 
item_size  varchar, 
item_price  double
                    )
                    WITH (
                        format = 'TEXTFILE',
                        skip_header_line_count = 1,
                        textfile_field_separator = ',',
                        external_location = 's3a://apresentacao//dev/bronze/order_items'
                    )
                    
Table minio.dev.order_items criada, file format csv 


""


In [95]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.order_items")


Execute SQL: SELECT * FROM minio.dev.order_items


,order_id,cust_name,item_name,item_size,item_price
0,ORD001,Alex,Espresso,,2.15
1,ORD002,Jordan,Hot Chocolate,Large,4.60
2,ORD003,Taylor,Espresso,,2.15
3,ORD004,Casey,Iced Tea,Medium,3.25
4,ORD005,Jamie,Sandwich Salami&Mozzarella,,5.50
...,...,...,...,...,...
461,ORD432,Fred,Mocha,Medium,4.00
462,ORD433,Gina,Sandwich Ham&Cheese,,5.60
463,ORD434,Hugh,Caramel Macchiato,Medium,4.20
464,ORD435,Iris,Cold Mocha,Large,4.60


In [96]:
sql = """
    SELECT ing.ing_id
         , ing.ing_name
         , ing.ing_price
         , r.recipe_id
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id
"""
recipe_ingredients = trinoProvider.get_result_pandas(sql)


Execute SQL: 
    SELECT ing.ing_id
         , ing.ing_name
         , ing.ing_price
         , r.recipe_id
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id



In [107]:
recipe_ingredients.head()

,ing_id,ing_name,ing_price,recipe_id
0,ING001,Espresso beans,12.0,CDR-CMO-LG
1,ING001,Espresso beans,12.0,CDR-CMO-MD
2,ING001,Espresso beans,12.0,CDR-CCF-LG
3,ING001,Espresso beans,12.0,CDR-CCF-MD
4,ING001,Espresso beans,12.0,HDR-WMO-LG


In [108]:
## Inserindo modificacao
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/bronze/recipe_ingredients/recipe_ingredients', data=recipe_ingredients, file_format='csv')

{'data': b'ing_id,ing_name,ing_price,recipe_id\nING001,Espresso beans,12.0,CDR-CMO-LG\nING001,Espresso beans,12.0,CDR-CMO-MD\nING001,Espresso beans,12.0,CDR-CCF-LG\nING001,Espresso beans,12.0,CDR-CCF-MD\nING001,Espresso beans,12.0,HDR-WMO-LG\nING001,Espresso beans,12.0,HDR-WMO-MD\nING001,Espresso beans,12.0,HDR-MOC-LG\nING001,Espresso beans,12.0,HDR-MOC-MD\nING001,Espresso beans,12.0,HDR-ESP\nING001,Espresso beans,12.0,HDR-CRM-LG\nING001,Espresso beans,12.0,HDR-CRM-MD\nING001,Espresso beans,12.0,HDR-FLT\nING001,Espresso beans,12.0,HDR-LAT-LG\nING001,Espresso beans,12.0,HDR-LAT-MD\nING001,Espresso beans,12.0,HDR-CAP-LG\nING001,Espresso beans,12.0,HDR-CAP-MD\nING002,Whole Milk,1.2,CDR-CMO-LG\nING002,Whole Milk,1.2,CDR-CMO-MD\nING002,Whole Milk,1.2,CDR-CCF-LG\nING002,Whole Milk,1.2,CDR-CCF-MD\nING002,Whole Milk,1.2,HDR-HCH-LG\nING002,Whole Milk,1.2,HDR-HCH-MD\nING002,Whole Milk,1.2,HDR-WMO-LG\nING002,Whole Milk,1.2,HDR-WMO-MD\nING002,Whole Milk,1.2,HDR-MOC-LG\nING002,Whole Milk,1.2,HDR-MO

In [109]:
## Criando tabela da transfomacao no trino
key_s3=f"{SCHEMA}/bronze/recipe_ingredients"
trinoProvider.create_table_and_metadata(df=recipe_ingredients, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA,  key_s3=key_s3, table='recipe_ingredients', separator=',', format_file='csv')

NAME:  ing_id  TYPE:  object
NAME:  ing_name  TYPE:  object
NAME:  ing_price  TYPE:  float64
NAME:  recipe_id  TYPE:  object

Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'
Schema dev exist

Execute SQL: SHOW TABLES FROM minio.dev LIKE 'recipe_ingredients'
Creating table minio.dev.recipe_ingredients ...

Execute SQL: CREATE TABLE IF NOT EXISTS minio.dev.recipe_ingredients (
                        ing_id  varchar, 
ing_name  varchar, 
ing_price  double, 
recipe_id  varchar
                    )
                    WITH (
                        format = 'TEXTFILE',
                        skip_header_line_count = 1,
                        textfile_field_separator = ',',
                        external_location = 's3a://apresentacao//dev/bronze/recipe_ingredients'
                    )
                    
Table minio.dev.recipe_ingredients criada, file format csv 


""


In [110]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.recipe_ingredients")


Execute SQL: SELECT * FROM minio.dev.recipe_ingredients


,ing_id,ing_name,ing_price,recipe_id
0,ING001,Espresso beans,12.00,CDR-CMO-LG
1,ING001,Espresso beans,12.00,CDR-CMO-MD
2,ING001,Espresso beans,12.00,CDR-CCF-LG
3,ING001,Espresso beans,12.00,CDR-CCF-MD
4,ING001,Espresso beans,12.00,HDR-WMO-LG
...,...,...,...,...
56,ING014,Lemons,1.50,CDR-LMN-MD
57,ING015,Ham,27.50,SNK-SHC
58,ING016,Salami,15.49,SNK-SSM
59,ING016,Salami,15.49,CDR-ICT-LG


In [115]:
sql = """
    SELECT *
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id
"""
trinoProvider.get_result_pandas(sql)


Execute SQL: 
    SELECT *
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id



,ing_id,ing_name,ing_weight,ing_meas,ing_price,row_id,recipe_id,ing_id,quantity
0,ING001,Espresso beans,1000,grams,12.00,45,CDR-CMO-LG,ING001,10
1,ING001,Espresso beans,1000,grams,12.00,42,CDR-CMO-MD,ING001,8
2,ING001,Espresso beans,1000,grams,12.00,40,CDR-CCF-LG,ING001,10
3,ING001,Espresso beans,1000,grams,12.00,38,CDR-CCF-MD,ING001,8
4,ING001,Espresso beans,1000,grams,12.00,27,HDR-WMO-LG,ING001,10
...,...,...,...,...,...,...,...,...,...
56,ING014,Lemons,5,units,1.50,52,CDR-LMN-MD,ING014,1
57,ING015,Ham,1000,grams,27.50,57,SNK-SHC,ING015,50
58,ING016,Salami,1000,grams,15.49,60,SNK-SSM,ING016,50
59,ING016,Salami,1000,grams,15.49,50,CDR-ICT-LG,ING016,15
